In [234]:
import json

# Replace 'Emails.json' with the name of your JSON file
json_file = 'Emails.json'

# Read the JSON file
with open(json_file, mode='r', encoding='utf-8') as f:
    data = json.load(f)

# Count the number of 'Body' keys
body_count = sum(1 for row in data if 'Body' in row)

print(f"Number of 'Body' keys: {body_count}")

Number of 'Body' keys: 12294


In [235]:
# Apply lowercasing to the 'Body' field
for row in data:
    if 'Body' in row:
        row['Body'] = row['Body'].lower()

In [236]:
import re

# Function to remove email headers
def remove_email_headers(text):
    # The regex pattern below matches common email headers
    pattern = r"(?:From|To|Subject|Date|Cc|Bcc):\s*[^\n]*\n"
    return re.sub(pattern, "", text, flags=re.IGNORECASE)

# Apply the remove_email_headers function to each email body
for row in data:
    if 'Body' in row:
        row['Body'] = remove_email_headers(row['Body'])

In [237]:
# Function to remove URLs
def remove_urls(text):
    url_pattern = re.compile(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    return url_pattern.sub('', text)

# Remove URLs from the 'Body' field
for row in data:
    if 'Body' in row:
        row['Body'] = remove_urls(row['Body'])

In [238]:
!pip install beautifulsoup4 -q

In [239]:
from bs4 import BeautifulSoup

# Remove HTML tags from the 'Body' field
for row in data:
    if 'Body' in row:
        soup = BeautifulSoup(row['Body'], 'html.parser')
        row['Body'] = soup.get_text()

<ipython-input-239-e640f458e775>:6: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(row['Body'], 'html.parser')


In [240]:
# Define a function to remove non-alphabetic characters
def remove_non_alphabetic(text):
    # Replace non-alphabetic characters with spaces
    return re.sub(r'[^a-zA-Z\s]', ' ', text)

# Iterate through the list of dictionaries and clean the 'Body' key
for row in data:
    if 'Body' in row:
        row['Body'] = remove_non_alphabetic(row['Body'])

In [241]:
# Print the 'Body' value of the first three rows
for i, row in enumerate(data):
    if i < 3:
        print(f"Row {i+1}:")
        print(row['Body'])
        print()
    else:
        break

Row 1:
you re signed up to receive a daily report of some notifications from your canvas account  below is the report for    mar 

submission comment  achraf ajrhourh             project    sp   csc       data engineering and visualization

tajjeeddine rachidi just made a new comment on the submission for achraf ajrhourh            for project    

click to view   

   	
update your notification settings    	


Row 2:
dear community  

we would like to remind you that we will go back from gmt   to gmt  that is  time on watches should be decreased by one hour starting sunday   th at  am 

we would also like to take this opportunity to wish you a blessed ramadan  may this holy month bring you prosperity  health  and peace 

ramadan mubarak kareem

best regards 



student government association


al akhawayn university

office  building      beneath athletics office


   


sga aui ma  mail
sga aui ma   





Row 3:
   	

 

 

thank you for joining the waitlist to build with gpt      



In [242]:
!pip install nltk -q

In [243]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.corpus import stopwords, wordnet
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('words')

# Load the custom stopwords from the text file
custom_stopwords = []
with open('uni_stopwords.txt', 'r') as file:
    for line in file:
        custom_stopwords.append(line.strip())

# Combine NLTK's built-in stopwords with custom stopwords
stopwords_list = stopwords.words('english') + custom_stopwords

# Loop to remove stopwords, non-English words, and append the modified text
english_words = set(nltk.corpus.words.words())

tokenized_data = []
for row in data:
    if 'Body' in row:
        tokens = nltk.word_tokenize(row['Body'])
        filtered_tokens = [word.lower() for word in tokens if (word.lower() not in stopwords_list and word.isalpha() and word.lower() in english_words)]
        tokenized_data.append({'Body': filtered_tokens})
    else:
        tokenized_data.append(row)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [244]:
from collections import Counter

def print_top_25_words(data):
    all_words = []
    for row in data:
        all_words.extend(row['Body'])
    counter = Counter(all_words)
    for word, count in counter.most_common(25):
        print(word, count)

print_top_25_words(tokenized_data)

community 2858
team 2589
join 2499
event 2395
form 2367
registration 2176
session 2139
aw 2113
program 2025
school 1974
assignment 1973
science 1945
semester 1907
meeting 1862
club 1833
today 1827
week 1825
room 1760
professor 1702
day 1628
campus 1599
project 1574
available 1494
model 1462
business 1403


In [245]:
from nltk.stem import WordNetLemmatizer

def lemmatize_data(data):
    lemmatizer = WordNetLemmatizer()
    lemmatized_data = []
    for row in data:
        if 'Body' in row:
            tokens = row['Body']
            lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
            lemmatized_data.append({'Body': lemmatized_tokens})
        else:
            lemmatized_data.append(row)
    return lemmatized_data

lemmatized_data = lemmatize_data(tokenized_data)

In [246]:
print_top_25_words(lemmatized_data)

community 2858
session 2770
team 2589
join 2499
day 2398
event 2395
form 2367
registration 2176
aw 2113
program 2025
school 1974
assignment 1973
class 1965
science 1945
semester 1907
meeting 1862
club 1833
today 1827
week 1825
room 1760
professor 1702
campus 1599
project 1574
available 1494
model 1462


In [247]:
def filter_data(data, min_len=3, max_len=15):
    filtered_data = []
    for row in data:
        if 'Body' in row:
            tokens = row['Body']
            filtered_tokens = [token for token in tokens if (len(token) >= min_len and len(token) <= max_len)]
            filtered_data.append({'Body': filtered_tokens})
        else:
            filtered_data.append(row)
    return filtered_data

filtered_data = filter_data(lemmatized_data, min_len=3, max_len=15)

In [248]:
def remove_infrequent_words(data, min_freq=5):
    all_words = []
    for row in data:
        if 'Body' in row:
            tokens = row['Body']
            all_words.extend(tokens)

    word_freq = Counter(all_words)
    filtered_data = []
    for row in data:
        if 'Body' in row:
            tokens = row['Body']
            filtered_tokens = [token for token in tokens if word_freq[token] >= min_freq]
            filtered_data.append({'Body': filtered_tokens})
        else:
            filtered_data.append(row)
    return filtered_data

cleaned_data = remove_infrequent_words(filtered_data, min_freq=7)

In [249]:
# Print the 'Body' value of the first three rows
for i, row in enumerate(cleaned_data):
    if i < 3:
        print(f"Row {i+1}:")
        print(row['Body'])
        print()
    else:
        break

Row 1:
['daily', 'report', 'canvas', 'report', 'mar', 'submission', 'comment', 'project', 'engineering', 'visualization', 'comment', 'submission', 'project', 'notification']

Row 2:
['community', 'remind', 'back', 'hour', 'opportunity', 'wish', 'blessed', 'holy', 'month', 'bring', 'prosperity', 'health', 'peace', 'government', 'association', 'beneath', 'athletics']

Row 3:
['joining', 'build', 'balance', 'capacity', 'demand', 'sending', 'gradually', 'previously', 'build', 'gain', 'priority', 'access', 'contribute', 'model', 'improve', 'everyone', 'process', 'different', 'based', 'capacity', 'expect', 'access', 'appreciate', 'interest', 'look', 'build', 'soon', 'suggest', 'getting', 'turbo', 'model', 'team', 'preview', 'chat', 'plus', 'subscriber', 'expect', 'severely', 'capacity', 'usage', 'cap', 'model', 'depending', 'demand', 'system', 'performance', 'market', 'san', 'united', 'manage']



In [250]:
print_top_25_words(cleaned_data)

community 2858
session 2770
team 2589
join 2499
day 2398
event 2395
form 2367
registration 2176
program 2025
school 1974
assignment 1973
class 1965
science 1945
semester 1907
meeting 1862
club 1833
today 1827
week 1825
room 1760
professor 1702
campus 1599
project 1574
available 1494
model 1462
business 1403


In [251]:
# Joining the tokenized data
def join_tokens(data):
    joined_data = []
    for row in data:
        if 'Body' in row:
            tokens = row['Body']
            joined_tokens = ' '.join(tokens)
            joined_data.append({'Body': joined_tokens})
        else:
            joined_data.append(row)
    return joined_data

joined_data = join_tokens(cleaned_data)

In [252]:
# Print the 'Body' value of the first three rows
for i, row in enumerate(joined_data):
    if i < 3:
        print(f"Row {i+1}:")
        print(row['Body'])
        print()
    else:
        break

Row 1:
daily report canvas report mar submission comment project engineering visualization comment submission project notification

Row 2:
community remind back hour opportunity wish blessed holy month bring prosperity health peace government association beneath athletics

Row 3:
joining build balance capacity demand sending gradually previously build gain priority access contribute model improve everyone process different based capacity expect access appreciate interest look build soon suggest getting turbo model team preview chat plus subscriber expect severely capacity usage cap model depending demand system performance market san united manage



In [253]:
def count_bodies(data):
    count = 0
    for row in data:
        if 'Body' in row:
            count += 1
    print(f'Total number of "Body" entries: {count}')

count_bodies(joined_data)

Total number of "Body" entries: 12294


In [254]:
!pip install gensim -q

In [255]:
# Checks the joined_data list for "<number>" in the "To: (Name)" field, and classifies each email as personal or not
def classify_personal(data):
    personal_data = []
    number_regex = re.compile('< \s*(\d+)\s* >')
    for row in data:
        if 'To: (Name)' in row:
            to_field = row['To: (Name)']
            to_field = number_regex.sub(r'\d+', to_field)
            row['is_personal'] = bool(re.search('^\d+@', to_field))
        else:
            row['is_personal'] = False
        personal_data.append(row)
    return personal_data

personal_data = classify_personal(joined_data)

In [256]:
from gensim import corpora
from gensim.models import LdaModel
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF

# Create a list of the known topics
known_topics = ['event', 'workshop', 'club', 'professional', 'jobs & internship', 'general']

# Create a dictionary mapping known topics to a list of keywords
topic_keywords = {
    'event': ['party', 'gathering', 'conference', 'event'],
    'workshop': ['training', 'course', 'seminar', 'workshop'],
    'club': ['group', 'organization', 'association', 'club', 'meeting'],
    'professional': ['networking', 'development','grading', 'graded', 'assignment', 'grade'],
    'jobs & internship': ['job', 'internship', 'employment', 'career'],
    'general': ['information', 'news', 'announcement']
}

# Create a list of documents from the 'Body' entries in the personal_data
documents = [row['Body'] for row in personal_data if 'Body' in row and not row['is_personal']]

def find_topic_by_keyword(body, topic_keywords):
    for topic, keywords in topic_keywords.items():
        if any(keyword in body for keyword in keywords):
            return topic
    return None

mapped_documents = []
unmapped_documents = []


# Map documents to topics using keywords
for document in documents:
    topic = find_topic_by_keyword(document, topic_keywords)
    if topic is not None:
        mapped_documents.append((document, topic))
    else:
        unmapped_documents.append(document)


# Train LDA and NMF models on unmapped documents
unmapped_documents = [doc for doc, _ in mapped_documents]

# Tokenize each document
tokenized_documents = [doc.split() for doc in unmapped_documents]

# Create a dictionary from the tokenized documents
dictionary = corpora.Dictionary(tokenized_documents)

# Create a bag-of-words representation of the documents
bow_corpus = [dictionary.doc2bow(doc) for doc in tokenized_documents]

In [257]:
# Train an LDA model on the bag-of-words corpus
num_topics = len(known_topics)
lda_model = LdaModel(bow_corpus, num_topics=num_topics, id2word=dictionary, passes=10)

In [258]:
from gensim.models.coherencemodel import CoherenceModel

# Compute LDA model coherence score
lda_coherence_model = CoherenceModel(model=lda_model, texts=tokenized_documents, dictionary=dictionary, coherence='c_v')
lda_coherence_score = lda_coherence_model.get_coherence()

print(f"LDA Coherence Score: {lda_coherence_score}")

LDA Coherence Score: 0.4492731103083732


In [259]:
# Train an NMF model on the TF-IDF representation of the documents
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(unmapped_documents)
nmf_model = NMF(n_components=num_topics, random_state=1).fit(tfidf_matrix)

In [260]:
def get_nmf_top_words(nmf_model, vectorizer, num_words=10):
    topic_words = []
    feature_names = vectorizer.get_feature_names_out()
    
    for topic_idx, topic in enumerate(nmf_model.components_):
        top_words_indices = topic.argsort()[:-num_words - 1:-1]
        top_words = [feature_names[i] for i in top_words_indices]
        topic_words.append(top_words)
        
    return topic_words

nmf_top_words = get_nmf_top_words(nmf_model, vectorizer)

from gensim.models.coherencemodel import CoherenceModel

# Compute NMF model coherence score
nmf_coherence_model = CoherenceModel(topics=nmf_top_words, texts=tokenized_documents, dictionary=dictionary, coherence='c_v')
nmf_coherence_score = nmf_coherence_model.get_coherence()

print(f"NMF Coherence Score: {nmf_coherence_score}")

NMF Coherence Score: 0.6876887876173748


In [261]:
!pip install corextopic -q

In [262]:
from corextopic import corextopic as ct

# Convert the tokenized documents back to strings
preprocessed_documents = [' '.join(doc) for doc in tokenized_documents]

# Vectorize the preprocessed data using TfidfVectorizer
vectorizer1 = TfidfVectorizer(max_df=0.95, min_df=2, stop_words='english')
tfidf_matrix1 = vectorizer1.fit_transform(preprocessed_documents)

# Now that the vectorizer is fitted, you can get the feature names
feature_names = vectorizer1.get_feature_names_out()

# Create a CorEx model with your predefined topics
num_topics = len(known_topics)
corex_model = ct.Corex(n_hidden=num_topics, words=feature_names, max_iter=100, verbose=False, seed=42)

# Fit the CorEx model to the vectorized data
corex_model.fit(tfidf_matrix1)

In [263]:
# Get the top words for each topic
def get_corex_top_words(corex_model, feature_names, num_words=10):
    topic_words = []
    for topic_idx, topic in enumerate(corex_model.get_topics()):
        top_words = [feature_names[word_idx] for word_idx, _, _ in topic[:num_words]]
        topic_words.append(top_words)
    return topic_words

corex_top_words = get_corex_top_words(corex_model, feature_names)

# Calculate the coherence score using Gensim's CoherenceModel
corex_coherence_model = CoherenceModel(topics=corex_top_words, texts=tokenized_documents, dictionary=dictionary, coherence='c_v')
corex_coherence_score = corex_coherence_model.get_coherence()

print(f"CorEx Coherence Score: {corex_coherence_score}")

NOTE: 'words' not provided to CorEx. Returning topics as lists of column indices
CorEx Coherence Score: 0.5516474376788097


In [264]:
!pip install numpy -q

In [265]:
import numpy as np

# Get the top words for each topic from the LDA model
def get_lda_top_words(lda_model, num_words=10):
    topic_words = []
    for topic_id in range(lda_model.num_topics):
        words = lda_model.show_topic(topic_id, topn=num_words)
        topic_words.append([w[0] for w in words])
    return topic_words

lda_top_words = get_lda_top_words(lda_model, num_words=10)

# Get the top words for each topic from the NMF model
def get_nmf_top_words(nmf_model, vectorizer, num_words=10):
    topic_words = []
    for topic_id in range(nmf_model.components_.shape[0]):
        word_ids = nmf_model.components_[topic_id].argsort()[:-num_words - 1:-1]
        topic_words.append([vectorizer.get_feature_names_out()[i] for i in word_ids])
    return topic_words

nmf_top_words = get_nmf_top_words(nmf_model, vectorizer)

# Combine the top words from LDA, NMF, and CorEx models for each topic
combined_top_words = []
for i, topic in enumerate(known_topics):
    print(f"Topic: {topic}")
    print("Top words from LDA model:")
    print(lda_top_words[i])
    print("Top words from NMF model:")
    print(nmf_top_words[i])
    print("Top words from CorEx model:")
    print(corex_top_words[i])
    combined_words = list(set(lda_top_words[i] + nmf_top_words[i] + corex_top_words[i]))
    combined_top_words.append(combined_words)
    print("Combined top words:")
    print(combined_words)
    print("\n")

Topic: event
Top words from LDA model:
['club', 'event', 'community', 'professor', 'science', 'meeting', 'place', 'join', 'semester', 'tomorrow']
Top words from NMF model:
['assignment', 'graded', 'review', 'notification', 'roll', 'call', 'attendance', 'grade', 'quiz', 'mar']
Top words from CorEx model:
['assignment', 'graded', 'notification', 'review', 'roll', 'caution', 'grade', 'attendance', 'quiz', 'exercise']
Combined top words:
['notification', 'assignment', 'semester', 'meeting', 'call', 'tomorrow', 'mar', 'join', 'grade', 'club', 'event', 'caution', 'science', 'place', 'professor', 'exercise', 'attendance', 'graded', 'community', 'quiz', 'roll', 'review']


Topic: workshop
Top words from LDA model:
['model', 'layer', 'registration', 'train', 'size', 'input', 'test', 'intermediate', 'history', 'rate']
Top words from NMF model:
['event', 'club', 'community', 'session', 'program', 'research', 'place', 'team', 'semester', 'sao']
Top words from CorEx model:
['enroll', 'machine', 'sk

In [266]:
# Print the topics and the top words in each topic
for idx, topic in lda_model.print_topics(-1):
    topic_name = known_topics[idx]
    print(f'{topic_name}: {topic}')

event: 0.023*"club" + 0.022*"event" + 0.014*"community" + 0.010*"professor" + 0.010*"science" + 0.010*"meeting" + 0.009*"place" + 0.009*"join" + 0.008*"semester" + 0.008*"tomorrow"
workshop: 0.026*"model" + 0.024*"layer" + 0.024*"registration" + 0.023*"train" + 0.022*"size" + 0.021*"input" + 0.021*"test" + 0.018*"intermediate" + 0.017*"history" + 0.016*"rate"
club: 0.051*"assignment" + 0.027*"graded" + 0.024*"due" + 0.021*"notification" + 0.018*"review" + 0.015*"report" + 0.014*"school" + 0.013*"group" + 0.012*"international" + 0.011*"social"
professional: 0.013*"meeting" + 0.010*"join" + 0.009*"housing" + 0.009*"form" + 0.008*"book" + 0.007*"campus" + 0.007*"group" + 0.007*"cash" + 0.006*"trip" + 0.006*"day"
jobs & internship: 0.007*"read" + 0.007*"machine" + 0.007*"caution" + 0.006*"organization" + 0.006*"model" + 0.006*"training" + 0.005*"team" + 0.005*"text" + 0.005*"language" + 0.005*"cloud"
general: 0.017*"session" + 0.011*"team" + 0.010*"program" + 0.008*"development" + 0.008*"e

In [267]:
json_file = 'Emails.json'

output_file = 'Email_add.json'

# Read the JSON file and store the email bodies in a list
with open(json_file, mode='r', encoding='utf-8') as f:
    data1 = json.load(f)

original_documents = [row['Body'] for row in data1 if 'Body' in row]

# Map the documents to the known topics
for i, document in enumerate(tokenized_documents):
    bow_vector = dictionary.doc2bow(document)
    topic_scores = lda_model[bow_vector]
    top_topic = max(topic_scores, key=lambda x: x[1])[0] + 1
    topic_name = known_topics[top_topic-1]

    # Add the topic and personal fields to the email data
    data1[i]['Category'] = topic_name
    data1[i]['Is_personal'] = bool(topic_name == 'personal')

    # Print if the top topic is 'personal'
    if topic_name == 'personal':
        print(f'Document {i+1}: {original_documents[i]}')
        print("The topic is personal.")

# Write the updated email data to a new JSON file
with open(output_file, mode='w', encoding='utf-8') as f:
    json.dump(data1, f, indent=2)

In [269]:
# Save the model
import pickle

with open('model.pkl', 'wb') as f:
    pickle.dump((lda_model, nmf_model, corex_model), f)